using LightGraphs
using GraphPlot
using GraphRecipes
using Plots
using Printf

In [ ]:
using LaTeXStrings

In [ ]:
# Requires the development version
include("../../dev/dice_library.jl")

In [ ]:
G8 = SimpleGraph(8)
add_edge!(G8, 1, 2)
add_edge!(G8, 2, 3)
add_edge!(G8, 3, 4)
add_edge!(G8, 4, 5)
add_edge!(G8, 5, 6)
add_edge!(G8, 6, 7)
add_edge!(G8, 7, 8)
add_edge!(G8, 1, 8)
add_edge!(G8, 1, 5)
add_edge!(G8, 2, 6)
add_edge!(G8, 3, 7)
add_edge!(G8, 4, 8)

p=graphplot(G8, nodesize = 0.05, nodeshape=:circle, curves=false, nodecolor=:black)

In [ ]:
G = G8
mtds = Dice.sine
#mtds = DyNN.piecewise
scale = 0.125/nv(G)
tmax = 125

Ksran = collect(0:0.025:4)
MM = length(Ksran)

Nrep = 10000

probs = zeros(MM)

for j in 1:MM
    print("\r$j")
    Ks = Ksran[j]

    count_best = 0

    for i in 1:Nrep
        V3 = Dice.get_initial(nv(G),(-0.15,0.55));
        V3t = Dice.propagate(G, mtds, Ks, scale/1, tmax, V3);
        fconf = Dice.extract_configuration(Dice.roundup(V3t[:, end]), 0)
        curcut = Dice.cut(G, fconf)
        if curcut == 10
            count_best += 1
        end
    end
    println("\nFor Ks = $Ks we've got $count_best successes out of $Nrep")
    probs[j] = count_best/Nrep
end
plot(Ksran, probs)

In [ ]:
# The dependence of the success probability on the anisotropy constant
gf = 24
tf = 22
xll = L"$K_s/K$"
yll = "Probability"
display(plot(Ksran, probs, linewidth = 5, guidefontsize = gf, tickfontsize = tf, legends = false,size=(400,400))) #, xlab = xll, ylab = yll)
savefig("psucc_ks.png")

In [ ]:
# Three dynamical regimes
# To get these plots, the code above should be changed manually to get V1t, V2t, V3t for different anisotropies
xll = "Time (arb.units)"
yll = L"$\theta/2\pi$"
gf = 20
tf = 18
lw = 3
p1=plot(1 .*(1:tmax), (V1t .+ 0.25)', legends = false, ylim = (-0.25, 0.75), guidefontsize = gf, tickfontsize = tf, linewidth = lw)
p2=plot(1 .*(1:tmax), (V2t .+ 0.25)', legends = false, ylim = (-0.25, 0.75), guidefontsize = gf, tickfontsize = tf, linewidth = lw)
p3=plot(1 .*(1:tmax), (V3t .+ 0.25)', legends = false, ylim = (-0.25, 0.75), guidefontsize = gf, tickfontsize = tf, linewidth = lw)
display(plot(p1, p2, p3, layout=(1,3), size = (1100, 600)))
savefig("three-regimes.png")

In [ ]:
MM = size(V1t)[2]

VD = zeros(1, MM)
HD = zeros(1, MM)

HI1  = zeros(1, MM)
HI2  = zeros(1, MM)

for i in 1:MM
    distri1 = Dice.roundup(V1t[:,i])
    distri2 = Dice.roundup(V2t[:,i])
    
    VD[i] = VectorD(V1t[:,i], V2t[:,i])
    
    conf1 = Dice.extract_configuration(distri1, 0.25)
    conf2 = Dice.extract_configuration(distri2, 0.25)
    
    HD[i] = Dice.HammingD(conf1, conf2)
    
    HI1[i] = Dice.cut(G, conf1)
    HI2[i] = Dice.cut(G, conf2)    
end

pvd = plot(1:MM, VD')
display(pvd)

phd = plot(1:MM, HD')
display(phd)

plot(1:MM, HI1', lc = :blue, label = false)
pc2 = plot!(1:MM, HI2', lc = :red, label = false)
display(pc2)

println("The pivot's best cut:  ", maximum(HI1), " its final value: ", HI1[end])
println("The probe's best cut:  ", maximum(HI2), " its final value: ", HI2[end])